In [ ]:
import numpy as np
import cv2
import keras.backend as K
import tensorflow as tf
from time import time
import matplotlib.pyplot as plt
from keras.applications.densenet import DenseNet121
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.models import Model

In [ ]:
'''
Note this code assumes you have two class model and the last layer of the model is softmax.
Heatmaps are created in the default resolution of (768,768).
The default size of model input is (224,224)

'''

global graph
graph = tf.get_default_graph()
def get_score_and_interm_output(cv2_input_image, DenseNetImageNet121_model):

     with graph.as_default():

        #define your tensor placeholders for, labels and images
        label_vector1 = tf.placeholder("float", [None, 2])

        cost = DenseNetImageNet121_model.output.op.inputs[0]*label_vector1

        # Get last convolutional layer gradients for generating gradCAM++ visualization
        target_conv_layer = DenseNetImageNet121_model.layers[-5].output
        target_conv_layer_grad = K.gradients(cost, target_conv_layer)[0]
        
        #Predicted probabilities
        pred = DenseNetImageNet121_model.layers[-1].output
        

        first_grad = []
        start1 = time() 
        for i in range(2):
            output = [0.0]*DenseNetImageNet121_model.layers[-1].output.get_shape().as_list()[1] #one-hot embedding for desired class activations
            #creating the output vector for the respective class
            output[i] = 1.0
            output = np.array(output)
            
            gradient_function = K.function([DenseNetImageNet121_model.layers[0].input,label_vector1], [target_conv_layer, target_conv_layer_grad, pred])

            [conv_output1, conv_first_grad, prediction] = gradient_function([cv2_input_image[None,:,:,:],output.reshape((1,-1))])
        
            first_grad.append(conv_first_grad[0])
            prediction = prediction[0,:]
        
        end1 = time()
        print('Time to calculate gradients is {0}'.format(end1-start1))
        return conv_output1, first_grad, prediction



def weights_all(score_grad,conv_output,prediction):
    
    temp1 = 0
    for j in range(2):
        temp1 += prediction[j]*score_grad[j]
        
        
    conv_first_grad = []
    for i in range(2):            
        conv_first_grad.append(prediction[i]*(score_grad[i] - temp1))
        
    temp2 = 0
    for j in range(2):
        temp2 += conv_first_grad[j]*score_grad[j]
        
    conv_second_grad = []
    for i in range(2):
        conv_second_grad.append(conv_first_grad[i]*(score_grad[i] - temp1) - prediction[i]*temp2)
        
    temp3 = 0
    for j in range(2):
        temp3 += conv_second_grad[j]*score_grad[j]
        
    conv_third_grad = []
    for i in range(2):
        conv_third_grad.append(conv_second_grad[i]*(score_grad[i] - temp1) - \
                                2*conv_first_grad[i]*temp2 - prediction[i]*temp3)
        
        
        
    weights_temp = []
    cams = []
    for i in range(2):
        global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[i].shape[2])), axis=0)

        alpha_num = conv_second_grad[i]
        alpha_denom = conv_second_grad[i]*2.0 + conv_third_grad[i]*global_sum.reshape((1,1,conv_first_grad[i].shape[2]))
        alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
        alphas = alpha_num/alpha_denom

        weights = np.maximum(conv_first_grad[i], 0.0)
        #normalizing the alphas
        """	
        alpha_normalization_constant = np.sum(np.sum(alphas, axis=0),axis=0)

        alphas /= alpha_normalization_constant.reshape((1,1,conv_first_grad[0].shape[2]))
        """

        alphas_thresholding = np.where(weights, alphas, 0.0)

        alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
        alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))


        alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[i].shape[2]))



        deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[i].shape[2])),axis=0)
        
        weights_temp.append(deep_linearization_weights)
        
        
        grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

        # Passing through ReLU
        cam = np.maximum(grad_CAM_map, 0)
        cam = cam / np.max(cam) # scale 0 to 1.0   

        cam = cv2.resize(cam, (224,224))
        # Passing through ReLU
        cam = np.maximum(grad_CAM_map, 0)
        cam = cam / np.max(cam) # scale 0 to 1.0    
        cam = cv2.resize(cam, (224,224))

        cams.append(cam)
        
    return np.array(cams)

def get_cam_picture(cams,dis_idx):
    #Create the class activation map.
    all_cams = []
    for m in dis_idx:
        grad_CAM_map = cv2.resize((cams[m]*-1.0) + 1.0, (768, 768), cv2.INTER_LINEAR)
        jetcam = cv2.applyColorMap(np.uint8(255 * grad_CAM_map), cv2.COLORMAP_JET)
        all_cams.append(jetcam)
    
    return all_cams



def save_fig(im_data,mp,path):
    dpi = 80

    height, width, nbands = im_data.shape


    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, interpolation='nearest')
    ax.imshow(mp/255.,alpha=0.3)#,cmap='jet')
#     ax.imshow(mp)

    ax.set(xlim=[0, width], ylim=[height, 0], aspect=1)

    fig.savefig(path, dpi=dpi, transparent=True)
    plt.close()
    
def cam_predict(img1,img,fname,shape,dest,class_names,model,threshold):
    
    img = img
    conv_outputs,class_score_grads,preds = get_score_and_interm_output(img1,model)
    dis_probs = {}
    dis_idx = []

    if(preds[1]>threshold):
        dis_idx.append(1)
        dis_probs[class_names[1]] = preds[1]
        pathology = class_names[1]


    dis_idx = np.array(dis_idx)
    cams = weights_all(class_score_grads,conv_outputs,preds)
    all_cams = get_cam_picture(cams,dis_idx)
    
    res = img

    #Changing the size of the image so that the heat map can be fitted on the dicom.
    #img = cv2.resize(img,(shape[1],shape[0]))
    img = cv2.resize(img,(768,768))
    for i in range(0,len(dis_probs)):    
        dis_name = class_names[dis_idx][i]
        path = dest + "{0}_{1}_heatmap_grad_cam_plus_plus.png".format(fname,dis_name)
        save_fig(img,cv2.resize(all_cams[i],(768,768)),path)

    return preds,res,all_cams

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def img_standardization(x):
    
#     x=cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x=x.astype('float16')/255.
#     return x
    return ((x-imagenet_mean)/imagenet_std)


def get_heatmaps(path,dest,class_names,model,threshold):
    start2 = time()
    img_name = path.split('/')[-1].strip('.png')
    x = cv2.imread(path)
    x = cv2.resize(x,(224,224))
    x1 = img_standardization(x)
    shape = x.shape
    preds,_,_ = cam_predict(x1,x,img_name,shape,dest,class_names,model,threshold)
    if preds[1] > threshold:
        print("Heatmaps Created",img_name)
    else:
        print("Heatmaps not created",img_name)
    print(preds)
    end2 = time()
    print('Time for entire process is {0}'.format(end2 - start2))
    return preds

# Example

In [ ]:
# Defining model
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')

preds = Dense(2,activation='softmax')(dense_model.output)
model = Model(dense_model.input,preds)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.load_weights('./Weights/Edema_vs_Healthy_pretrained_CHAI_TBvsNTB.hdf5')

In [ ]:
import glob
all_pngs=glob.glob('./Edema_Images_ToBeAnnotated/*/*')

In [ ]:
# Please define class names as per your model
cls = ['Healthy',
'Edema']
cls = np.array(cls)

for idx, i in enumerate(all_pngs):
    if idx%100==0:
        get_heatmaps(i,'./Heatmaps_0.11/',cls,model,threshold = 0.11)
        print (idx)

In [ ]:
all_pngs[0]

In [ ]:
import numpy as np
from keras.utils import to_categorical

In [ ]:
to_categorical([0,1])